Authentification

In [ ]:
# pip install ccxt
import ccxt
import os
from dotenv import load_dotenv

load_dotenv()

wallet_address = os.getenv('HYPERLIQUID_WALLET_ADDRESS')
private_key = os.getenv('HYPERLIQUID_PRIVATE_KEY')

dex = ccxt.hyperliquid({
    "walletAddress": wallet_address,
    "privateKey": private_key,
})

In [ ]:
dex.fetch_balance()

Market order

In [ ]:
symbol = "ETH/USDC:USDC"
order_type = "market"
side = "buy"
amount = 0.0023
price = dex.load_markets()[symbol]["info"]["midPx"] # get current price for symbol asset
# HL market orders require price to calculate the max slippage price. Default slippage can be set in options(default is 5%).
dex.create_order(symbol, order_type, side, amount, price=price)

In [ ]:
# Get current price for all assets
dex.load_markets() 

In [ ]:
symbols = ["ETH/USDC:USDC"]
# Fetch open positions for specified symbols
dex.fetch_positions(symbols)

Close position

In [ ]:
symbol = "ETH/USDC:USDC"
order_type = "market"
side = "sell"
amount = dex.fetch_positions([symbol])[0]["contracts"] # Check total amount of symbol in positions
price = dex.load_markets()[symbol]["info"]["midPx"]

dex.create_order(symbol, order_type, side, amount, price=price, params={"reduceOnly":True})


Limit order

In [ ]:
symbol = "ETH/USDC:USDC"
order_type = "limit"
side = "buy"
amount = 0.01
price = 4300

order = dex.create_order(symbol, order_type, side, amount, price=price)
print(order)

In [ ]:
symbol = "ETH/USDC:USDC"
order_id = order["id"]
# Only work for limit order, market order fails
dex.cancel_order(order_id, symbol)

In [ ]:
dex.fetch_open_orders("ETH/USDC:USDC")

Set leverage and margin mode

In [ ]:
symbol = "ETH/USDC:USDC"
margin_mode = "cross" # cross
leverage = 2

dex.set_margin_mode(margin_mode, symbol, params={"leverage": leverage})


In [ ]:
symbol = "ETH/USDC:USDC"
leverage = 1
dex.set_leverage(leverage, symbol)
# dex.set_leverage(leverage, symbol, params={"marginMode": "isolated"})

Set TP SL

In [ ]:
# Main order triggers first
symbol = "ETH/USDC:USDC"
order_type = "market"
side = "buy"
amount = 0.0023
price = dex.load_markets()[symbol]["info"]["midPx"]

dex.create_order(symbol, order_type, side, amount, price=price)

In [ ]:
symbol = "ETH/USDC:USDC"
order_type = "market"
side = "sell"
price = dex.load_markets()[symbol]["info"]["midPx"]

amount = dex.fetch_positions([symbol])[0]["contracts"]
# amount = 0.005
tp_price = 4545
# Notes: price is required to entry as slippage protection
dex.create_order(symbol, order_type, side, amount, price=price, params={"takeProfitPrice": tp_price, "reduceOnly": True})

In [ ]:
symbol = "ETH/USDC:USDC"
order_type = "market"
side = "sell"
price = dex.load_markets()[symbol]["info"]["midPx"]

amount = dex.fetch_positions([symbol])[0]["contracts"]
sl_price = 4538
# According to the exchange requirement, TP or SL order is created seperately
order = dex.create_order(symbol, order_type, side, amount, price=price, params={"stopLossPrice": sl_price, "reduceOnly": True})

In [ ]:
dex.fetch_order(order["id"], symbol)

Stop market order

In [ ]:
symbol = "ETH/USDC:USDC"
order_type = "market"
side = "buy"
price = dex.load_markets()[symbol]["info"]["midPx"]
amount = 0.05
stop_price = 4200

dex.create_order(symbol, order_type, side, amount, price=price, params={"stopPrice": stop_price})